<a href="https://colab.research.google.com/github/alwayssubhankar/resume/blob/main/subhankar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee, geemap
ee.Authenticate()
ee.Initialize(project = 'ee-subhankarbarman38') # Replace with your own project id.

In [ ]:
landsat5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
  .filter(ee.Filter.lt('CLOUD_COVER', 5))\
  .filter(ee.Filter.eq('WRS_PATH', 89))\
  .filter(ee.Filter.eq('WRS_ROW', 78))

In [ ]:
landsat7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')\
  .filter(ee.Filter.lt('CLOUD_COVER', 5))\
  .filter(ee.Filter.eq('WRS_PATH', 89))\
  .filter(ee.Filter.eq('WRS_ROW', 78))

In [ ]:
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
  .filter(ee.Filter.lt('CLOUD_COVER', 5))\
  .filter(ee.Filter.eq('WRS_PATH', 89))\
  .filter(ee.Filter.eq('WRS_ROW', 78))

In [ ]:
mean1990 = landsat5.filterDate('1990', '1991').mean()
mean1995 = landsat5.filterDate('1995', '1996').mean()
mean2000 = landsat7.filterDate('2000', '2001').mean()
mean2005 = landsat5.filterDate('2005', '2006').mean()
mean2010 = landsat5.filterDate('2010', '2011').mean()
mean2015 = landsat8.filterDate('2015', '2016').mean()
mean2020 = landsat8.filterDate('2020', '2021').mean()

In [ ]:
# Landsat 5 & 7: SR_B2 and SR_B4
ndwi1990 = mean1990.normalizedDifference(['SR_B2', 'SR_B4'])
ndwi1995 = mean1995.normalizedDifference(['SR_B2', 'SR_B4'])
ndwi2000 = mean2000.normalizedDifference(['SR_B2', 'SR_B4'])
ndwi2005 = mean2005.normalizedDifference(['SR_B2', 'SR_B4'])
ndwi2010 = mean2010.normalizedDifference(['SR_B2', 'SR_B4'])

# Landsat 8: SR_B3 and SR_B5
ndwi2015 = mean2015.normalizedDifference(['SR_B3', 'SR_B5'])
ndwi2020 = mean2020.normalizedDifference(['SR_B3', 'SR_B5'])

In [ ]:
# ndwi2015 = mean2015.normalizedDifference(['SR_B3', 'SR_B5'])
# ndwi2020 = mean2020.normalizedDifference(['SR_B3', 'SR_B5'])

In [ ]:
Map=geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
Map.addLayer(ndwi1990, {'min': -0.2, 'max': 0.1}, 'NDWI 1990', False)
Map.addLayer(ndwi1995, {'min': -0.2, 'max': 0.1}, 'NDWI 1995', False)
Map.addLayer(ndwi2000, {'min': -0.2, 'max': 0.1}, 'NDWI 2000', False)
Map.addLayer(ndwi2005, {'min': -0.2, 'max': 0.1}, 'NDWI 2005', False)
Map.addLayer(ndwi2010, {'min': -0.2, 'max': 0.1}, 'NDWI 2010', False)
Map.addLayer(ndwi2015, {'min': -0.2, 'max': 0.1}, 'NDWI 2015', False)
Map.addLayer(ndwi2020, {'min': -0.2, 'max': 0.1}, 'NDWI 2020', False)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
binary1990 = ndwi1990.gt(0)
binary1995 = ndwi1995.gt(0)
binary2000 = ndwi2000.gt(0)
binary2005 = ndwi2005.gt(0)
binary2010 = ndwi2010.gt(0)
binary2015 = ndwi2015.gt(0)
binary2020 = ndwi2020.gt(0)

In [ ]:
Map.addLayer(binary1990, {'min': 0, 'max': 1}, 'Binary 1990', False)
Map.addLayer(binary1995, {'min': 0, 'max': 1}, 'Binary 1995', False)
Map.addLayer(binary2000, {'min': 0, 'max': 1}, 'Binary 2000', False)
Map.addLayer(binary2005, {'min': 0, 'max': 1}, 'Binary 2005', False)
Map.addLayer(binary2010, {'min': 0, 'max': 1}, 'Binary 2010', False)
Map.addLayer(binary2015, {'min': 0, 'max': 1}, 'Binary 2015', False)
Map.addLayer(binary2020, {'min': 0, 'max': 1}, 'Binary 2020', False)
Map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
change1995 = binary1990.subtract(binary1995).multiply(6)
change2000 = binary1995.subtract(binary2000).multiply(5)
change2005 = binary2000.subtract(binary2005).multiply(4)
change2010 = binary2005.subtract(binary2010).multiply(3)
change2015 = binary2010.subtract(binary2015).multiply(2)
change2020 = binary2010.subtract(binary2020).multiply(1)

In [ ]:
ed = change1995.add(change2000).add(change2005).add(change2010).add(change2015).add(change2020).rename('ed')
ed_masked = ed.mask(ed.neq(0))

In [ ]:
Map.addLayer(ed_masked, {'min': -6, 'max': 6, 'palette': ['red', 'white', 'darkgreen']}, 'ED', False)
Map


Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [ ]:
roi=Map.user_roi

In [ ]:
counts = ed.reduceRegion(
  reducer = ee.Reducer.frequencyHistogram(),
  geometry = roi,
  scale = 30)
counts.get('ed').getInfo()


{'-1': 14.125490196078431,
 '-2': 14,
 '-3': 2,
 '-4': 134,
 '-5': 117,
 '-6': 76,
 '0': 1760.5411764705896,
 '1': 155.3764705882353,
 '2': 123.87843137254902,
 '3': 1,
 '4': 17.627450980392155,
 '5': 2.1254901960784314}